In [1]:
from datetime import date

import dash
import dash_bootstrap_components as dbc
from dash import dcc
from dash import html
from dash.dependencies import Input, Output, State
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd 
from plotly.offline import init_notebook_mode,iplot,plot
init_notebook_mode(connected=True)

In [2]:
app = dash.Dash(external_stylesheets=[dbc.themes.FLATLY])

In [3]:
df = pd.read_csv('Data/covid_19_data.csv',parse_dates=['Last Update',"ObservationDate"])
df.rename(columns={'ObservationDate':'Date_ob', 'Country/Region':'Country'}, inplace=True)


df_confirm = pd.read_csv("Data/time_series_covid_19_confirmed.csv")
df_recover = pd.read_csv("Data/time_series_covid_19_recovered.csv")
df_death = pd.read_csv("Data/time_series_covid_19_deaths.csv")

df_confirm.rename(columns={'Country/Region':'Country'}, inplace=True)
df_recover.rename(columns={'Country/Region':'Country'}, inplace=True)
df_death.rename(columns={'Country/Region':'Country'}, inplace=True)

df_gender = pd.read_csv('gender_dataset.csv')
df_gender.rename(columns={'Deaths (% male)':'male_deaths',
                          'Deaths (% female)':'female_deaths',
                          'Cases (% male)':'male_cases',
                          'Cases (% female)':'female_cases',}, inplace=True)
df_gender['male_deaths'] = df_gender['male_deaths'].str.rstrip('%').astype('float') / 100.0
df_gender['female_deaths'] = df_gender['female_deaths'].str.rstrip('%').astype('float') / 100.0
df_gender['male_cases'] = df_gender['male_cases'].str.rstrip('%').astype('float') / 100.0
df_gender['female_cases'] = df_gender['female_cases'].str.rstrip('%').astype('float') / 100.0

In [4]:
df.head()

,SNo,Date_ob,Province/State,Country,Last Update,Confirmed,Deaths,Recovered
0,1,2020-01-22,Anhui,Mainland China,2020-01-22 17:00:00,1.0,0.0,0.0
1,2,2020-01-22,Beijing,Mainland China,2020-01-22 17:00:00,14.0,0.0,0.0
2,3,2020-01-22,Chongqing,Mainland China,2020-01-22 17:00:00,6.0,0.0,0.0
3,4,2020-01-22,Fujian,Mainland China,2020-01-22 17:00:00,1.0,0.0,0.0
4,5,2020-01-22,Gansu,Mainland China,2020-01-22 17:00:00,0.0,0.0,0.0


In [5]:
df_gender.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 16 columns):
 #   Column                                                       Non-Null Count  Dtype  
---  ------                                                       --------------  -----  
 0   Country code                                                 199 non-null    object 
 1   Country                                                      204 non-null    object 
 2   Case & death data by sex?                                    204 non-null    object 
 3   Cases date                                                   140 non-null    object 
 4   Cases where sex-disaggregated data is available              140 non-null    float64
 5   male_cases                                                   140 non-null    float64
 6   female_cases                                                 140 non-null    float64
 7   Deaths date                                                  109 non-null    obj

In [6]:
df_gender = df_gender.dropna(subset=['male_cases', 'female_cases'])

In [7]:
df["year"] = df["Date_ob"].dt.year

In [8]:
df["month"] = df["Date_ob"].dt.month

In [9]:
df.head()

,SNo,Date_ob,Province/State,Country,Last Update,Confirmed,Deaths,Recovered,year,month
0,1,2020-01-22,Anhui,Mainland China,2020-01-22 17:00:00,1.0,0.0,0.0,2020,1
1,2,2020-01-22,Beijing,Mainland China,2020-01-22 17:00:00,14.0,0.0,0.0,2020,1
2,3,2020-01-22,Chongqing,Mainland China,2020-01-22 17:00:00,6.0,0.0,0.0,2020,1
3,4,2020-01-22,Fujian,Mainland China,2020-01-22 17:00:00,1.0,0.0,0.0,2020,1
4,5,2020-01-22,Gansu,Mainland China,2020-01-22 17:00:00,0.0,0.0,0.0,2020,1


In [10]:
d={"2020-01":0,"2020-02":1,"2020-03":2,"2020-04":3,"2020-05":4,"2020-06":5,"2020-07":6,"2020-08":7,"2020-09":8,"2020-10":9,
  "2020-11":10,"2020-12":11,"2021-01":12,"2021-02":13,"2021-03":14,"2021-04":15,"2021-05":16}

In [11]:
# get total cases in country
#cases = df.groupby(["Country"]).sum().reset_index()
cases = df.copy()
cases["Date_new"] =pd.to_datetime(cases["month"].astype("str")+ "/" + cases["year"].astype("str"))
cases["Date_new"] = cases['Date_new'].dt.date.apply(lambda x: x.strftime('%Y-%m'))
cases["Date"] = cases["Date_new"].map(d)
#cases = cases.sort_values("Date_new")
cases

,SNo,Date_ob,Province/State,Country,Last Update,Confirmed,Deaths,Recovered,year,month,Date_new,Date
0,1,2020-01-22,Anhui,Mainland China,2020-01-22 17:00:00,1.0,0.0,0.0,2020,1,2020-01,0
1,2,2020-01-22,Beijing,Mainland China,2020-01-22 17:00:00,14.0,0.0,0.0,2020,1,2020-01,0
2,3,2020-01-22,Chongqing,Mainland China,2020-01-22 17:00:00,6.0,0.0,0.0,2020,1,2020-01,0
3,4,2020-01-22,Fujian,Mainland China,2020-01-22 17:00:00,1.0,0.0,0.0,2020,1,2020-01,0
4,5,2020-01-22,Gansu,Mainland China,2020-01-22 17:00:00,0.0,0.0,0.0,2020,1,2020-01,0
...,...,...,...,...,...,...,...,...,...,...,...,...
306424,306425,2021-05-29,Zaporizhia Oblast,Ukraine,2021-05-30 04:20:55,102641.0,2335.0,95289.0,2021,5,2021-05,16
306425,306426,2021-05-29,Zeeland,Netherlands,2021-05-30 04:20:55,29147.0,245.0,0.0,2021,5,2021-05,16
306426,306427,2021-05-29,Zhejiang,Mainland China,2021-05-30 04:20:55,1364.0,1.0,1324.0,2021,5,2021-05,16
306427,306428,2021-05-29,Zhytomyr Oblast,Ukraine,2021-05-30 04:20:55,87550.0,1738.0,83790.0,2021,5,2021-05,16


In [12]:
last_day=["2020-01-31","2020-02-29","2020-03-31","2020-04-30","2020-05-31","2020-06-30","2020-07-31","2020-08-31",
         "2020-09-30","2020-10-31","2020-11-30","2020-12-31","2021-01-31","2021-02-28","2021-03-31","2021-04-30","2021-05-29"]
len(last_day)

17

In [13]:
cases[cases["Date_ob"]=="2021-05-29"].groupby("Country").sum().reset_index()

,Country,SNo,Confirmed,Deaths,Recovered,year,month,Date
0,Afghanistan,305665,70111.0,2899.0,57281.0,2021,5,16
1,Albania,305666,132297.0,2449.0,129215.0,2021,5,16
2,Algeria,305667,128456.0,3460.0,89419.0,2021,5,16
3,Andorra,305668,13693.0,127.0,13416.0,2021,5,16
4,Angola,305669,34180.0,757.0,27646.0,2021,5,16
...,...,...,...,...,...,...,...,...
190,Vietnam,305832,6908.0,47.0,2896.0,2021,5,16
191,West Bank and Gaza,305833,307838.0,3492.0,300524.0,2021,5,16
192,Yemen,305834,6731.0,1319.0,3399.0,2021,5,16
193,Zambia,305835,94751.0,1276.0,91594.0,2021,5,16


In [14]:
cases

,SNo,Date_ob,Province/State,Country,Last Update,Confirmed,Deaths,Recovered,year,month,Date_new,Date
0,1,2020-01-22,Anhui,Mainland China,2020-01-22 17:00:00,1.0,0.0,0.0,2020,1,2020-01,0
1,2,2020-01-22,Beijing,Mainland China,2020-01-22 17:00:00,14.0,0.0,0.0,2020,1,2020-01,0
2,3,2020-01-22,Chongqing,Mainland China,2020-01-22 17:00:00,6.0,0.0,0.0,2020,1,2020-01,0
3,4,2020-01-22,Fujian,Mainland China,2020-01-22 17:00:00,1.0,0.0,0.0,2020,1,2020-01,0
4,5,2020-01-22,Gansu,Mainland China,2020-01-22 17:00:00,0.0,0.0,0.0,2020,1,2020-01,0
...,...,...,...,...,...,...,...,...,...,...,...,...
306424,306425,2021-05-29,Zaporizhia Oblast,Ukraine,2021-05-30 04:20:55,102641.0,2335.0,95289.0,2021,5,2021-05,16
306425,306426,2021-05-29,Zeeland,Netherlands,2021-05-30 04:20:55,29147.0,245.0,0.0,2021,5,2021-05,16
306426,306427,2021-05-29,Zhejiang,Mainland China,2021-05-30 04:20:55,1364.0,1.0,1324.0,2021,5,2021-05,16
306427,306428,2021-05-29,Zhytomyr Oblast,Ukraine,2021-05-30 04:20:55,87550.0,1738.0,83790.0,2021,5,2021-05,16


### Navigation Tab

In [15]:
navbar = dbc.NavbarSimple(
                    brand="",
                    color="primary",
                    dark=True,
                    )

### Cards for graphs

In [16]:
card_content_line_plot = [
    dbc.CardHeader("Trends in cases, deaths and recovered for a given country"),
    dbc.CardBody(
        [
            html.H5("Please choose the country", className="card-title"),
            dcc.Dropdown(id='drop_down_line_plot_0',
                         options=[{'label': i, 'value': i} for i in df['Country'].unique()],
                         value= df['Country'].loc[0]
                        ),
            dcc.Graph(id = 'graph_line_plot_0',style={'width': '90', 'height': '90'}, figure={}),
        ]
    ),
]

In [17]:
dict(zip(cases[["Date_new",'Date']].drop_duplicates()["Date"],cases[["Date_new",'Date']].drop_duplicates()["Date_new"]))

{0: '2020-01',
 1: '2020-02',
 2: '2020-03',
 3: '2020-04',
 4: '2020-05',
 5: '2020-06',
 6: '2020-07',
 7: '2020-08',
 8: '2020-09',
 9: '2020-10',
 10: '2020-11',
 11: '2020-12',
 12: '2021-01',
 13: '2021-02',
 14: '2021-03',
 15: '2021-04',
 16: '2021-05'}

In [18]:
card_content_map = [
    dbc.CardHeader("Cumulative Confirmed Cases Worldwide"),
    dbc.CardBody(
        [
            dcc.Graph(id = 'graph_map', style={'width': '300', 'height': '80'},figure={}),

#then in the Slider
dcc.Slider(id = 'slider_0',
               min=cases["Date"].min(), #the first date
               max=cases["Date"].max(), #the last date
               value=8, #default: the first
               #marks={str(date): date for date in cases['Date'].unique()})
               marks = {0: '2020-01',
                         2: '2020-03',
                         4: '2020-05',
                         6: '2020-07',
                         8: '2020-09',
                         10: '2020-11',
                         12: '2021-01',
                         14: '2021-03',
                         16: '2021-05'})
        ]
    ),
]
#‘style’:{‘transform’:‘rotate(-90deg)’, ‘font-size’:‘8px’}

In [19]:
card_content_bar = [
    dbc.CardHeader("Total Death for the 5 Highest Countries Worldwide"),
    dbc.CardBody(
        [
            dcc.Graph(id = 'graph_bar', style={'width': '90%', 'height': '80%'},figure={})
        ]
    ),
    
    dbc.CardFooter("."),

]
#‘style’:{‘transform’:‘rotate(-90deg)’, ‘font-size’:‘8px’}

In [20]:
card_content_table =[
    dbc.CardHeader("Percantage of Male Confirmed Cases vs. Female Confirmed Cases"),
    dbc.CardBody(
        [
            html.H5("Please choose the country", className="card-title_map"),
            
                        dcc.Dropdown(id='drop_down_pie_plot',
                         options=[{'label': i, 'value': i} for i in df_gender['Country'].unique()],
                         value= df_gender['Country'].loc[0]
                        ),
            
#             dcc.Dropdown(id='values', value='Deaths', 
#                          options=[{'value': x, 'label': x} for x in ['Deaths', 'Cases']],
#                          clearable=False),
   
            dcc.Graph(id="pie_chart",figure={}),
        ]
    ),
]

## Style

In [21]:
# the style arguments for the main content page.
CONTENT_STYLE = {
    'margin-left': '25%',
    'margin-right': '5%',
    'top': 0,
    'padding': '20px 10px'
}

TEXT_STYLE = {
    'textAlign': 'center',
    'color': '#191970'
}

CARD_TEXT_STYLE = {
    'textAlign': 'center',
    'color': '#0074D9'
}


### CallBacks (Graphs)

In [22]:
@app.callback(
    Output(component_id='graph_line_plot_0', component_property='figure'),
    Input(component_id='drop_down_line_plot_0', component_property='value')
)
def display_input(value):
    title = 'Time Series Data for Cumulative Cases, Death and Recovered Rates for ' + value
    df_for_value = df[df['Country'] == value]
    confirm = df_for_value.groupby('Date_ob').sum()['Confirmed'].reset_index()
    death = df_for_value.groupby('Date_ob').sum()['Deaths'].reset_index()
    recover = df_for_value.groupby('Date_ob').sum()['Recovered'].reset_index()
    fig = go.Figure()
    # Create and style traces
    fig.add_trace(go.Scatter(x=confirm['Date_ob'], 
                             y=confirm['Confirmed'],
                             mode='lines+markers',
                             name='Confirmed',
                             line=dict(color='blue', width=2)
                            ))
    fig.add_trace(go.Scatter(x=death['Date_ob'], 
                             y=death['Deaths'],
                             mode='lines+markers',
                             name='Deaths',
                             line=dict(color='Red', width=2)
                            ))
    fig.add_trace(go.Scatter(x=recover['Date_ob'], 
                             y=recover['Recovered'],
                             mode='lines+markers',
                             name='Recovered',
                             line=dict(color='Green', width=2)
                            ))
    fig.update_layout(
        title='Worldwide Corona Virus Cases',
        xaxis_tickfont_size=14,
        yaxis=dict(
            title='Number of Cases',
            titlefont_size=16,
            tickfont_size=14,
        ),
        legend=dict(
            x=0,
            y=1.0,
            bgcolor='rgba(255, 255, 255, 0)',
            bordercolor='rgba(255, 255, 255, 0)'
        )
    )
    return fig

In [23]:
@app.callback(
    Output(component_id='graph_map', component_property='figure'),
    Input(component_id='slider_0', component_property='value')
)
def display_input(value_slider):
    data = dict(
        type = 'choropleth',
        colorscale = 'Viridis',
        locations = cases[cases["Date_ob"]==last_day[value_slider]].groupby("Country").sum().reset_index()['Country'],
        locationmode = "country names",
        z = cases[cases["Date_ob"]==last_day[value_slider]].groupby("Country").sum().reset_index()['Confirmed'],
        text = cases[cases["Date_ob"]==last_day[value_slider]].groupby("Country").sum().reset_index()['Country'],
        colorbar = {'title' : 'Cumulative Confirmed Cases'},
      )
    layout = dict(title = 'Cumulative Cases',
            #  geo = dict(projection = {'type':'mercator'})
                 )
    fig = go.Figure(data = [data],layout = layout)
    fig.update_layout(
    margin=dict(l=20, r=20, t=30, b=10))

    return fig

In [24]:
@app.callback(
    Output(component_id='graph_bar', component_property='figure'),
    Input(component_id='slider_0', component_property='value')
)
def display_input(value_slider):
    d = cases[cases["Date_ob"]== last_day[value_slider]].groupby("Country").sum().reset_index()
    d = d.sort_values(by=['Deaths'],ascending=False).head(5)

    fig = go.Figure(go.Bar(
            y=d["Country"],
            x=d["Deaths"],
            orientation='h'))
    fig.update_layout(        yaxis=dict(
            title='Country',
            titlefont_size=16,
            tickfont_size=14,
        ),
            xaxis=dict(
            title='Number of Deaths',
            titlefont_size=16,
            tickfont_size=14,
        ))
    return fig     

In [25]:
@app.callback(
    Output(component_id='pie_chart', component_property='figure'),
    [Input(component_id='drop_down_pie_plot', component_property='value'),
   #  Input(component_id="values", component_property="value")
    ]
)

def display_input(drop_down):
    title = 'Gender comparison of total ' + drop_down
    
    df_for_value = df_gender[df_gender['Country'] == drop_down]

    df_for_cases = df_for_value[["male_cases","female_cases"]]
    temp = df_for_cases.melt(var_name='source')

    fig = px.pie(temp, values="value", names="source",color='source',
             color_discrete_map={'male_cases':'babyblue',
                                 'female_cases':'darkblue'})


    return fig

### Rows

In [26]:
# row_0 = dbc.Row([
#     dbc.Col(
#         dbc.Card(
#             [dbc.CardBody([html.H3('Confirmed : 125M',  className='card-title'),
#                            html.P('card_text_1', style="info"),])], outline="info"),md=4),
#     dbc.Col(
#         dbc.Card(
#             [dbc.CardBody([html.H3('Death : 125M', className='card-title'),
#                            html.P('Sample text.', style="primary"),]),], outline="primary"),md=4),
#     dbc.Col(
#         dbc.Card(
#             [dbc.CardBody([html.H3('Recovered : 456M ', className='card-title'),
#                            html.P('Sample text.', style="success"),]),], outline="success"),md=4)
# ])

In [27]:
row_1 = dbc.Row(
        [
        dbc.Col(dbc.Card(card_content_map, color="info", outline=True),width=8),
        dbc.Col(dbc.Card(card_content_bar, color="info", outline=True),width=4)    
        ],
        className="mb-4",
)


In [28]:
row_2 = dbc.Row(
        [
            dbc.Col(dbc.Card(card_content_line_plot, color="info", outline=True),width=7),
            dbc.Col(dbc.Card(card_content_table, color="info", outline=True),width=5)
        ],
        className="mb-4",
)


### App

In [29]:
#app.layout = html.Div([navbar, row_1, row_2])
app.layout = html.Div([    
    html.H1("Covid19 Dashboard", style={'text-align': 'center'}),
    html.Br(),
    navbar,
 row_1,row_2])

In [ ]:
if __name__ == '__main__': 
    app.run_server(debug = False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Oct/2021 10:23:30] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2021 10:23:31] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2021 10:23:31] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2021 10:23:31] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [27/Oct/2021 10:23:31] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [27/Oct/2021 10:23:31] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 304 -
127.0.0.1 - - [27/Oct/2021 10:23:31] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [27/Oct/2021 10:23:31] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2021 10:23:31] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2021 10:23:31] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2021 10:23:32] "POST /_dash-